# Test results analysis

* Download results

In [1]:
from paramiko import SSHClient
from scp import SCPClient

USER='hadoop'
HOST='172.16.4.135'
PASSWD='hadoop'
REMOTE_WORKING_DIR = 'bloom'
REMOTE_LOGS=f'/home/{USER}/{REMOTE_WORKING_DIR}/logs'
LOGS='data/logs'

client = SSHClient()
client.load_system_host_keys()
client.connect(hostname=HOST, username=USER, password=PASSWD)

scp = SCPClient(client.get_transport())
#scp.get(REMOTE_LOGS, recursive=True, local_path='data/')
scp.close()
client.close()

* Load data

In [10]:
import json
import glob
log_filename = glob.glob("data/logs/test_log*.json")[-1]

logs = {}
with open(log_filename) as js:
    logs = json.load(js)

with open(log_filename, 'w') as js:
   json.dump(logs, js, indent=2)

In [64]:
from datetime import datetime 
import numpy as np
import pandas as pd

start = datetime.strptime(logs['start'], '%d-%m-%Y_%H.%M.%S')
stop = datetime.strptime(logs['stop'], '%d-%m-%Y_%H.%M.%S')
total_time = stop - start
print(f'Total execution time: {total_time}')

with_bloom_filter = logs['tests']['WithBloomFilters']
with_indexes = logs['tests']['WithIndexes']

wall_time = {'withBloomFilter':0,'withIndexes':0}
for test in with_bloom_filter:
    wall_time['withBloomFilter'] += np.sum(pd.DataFrame(with_bloom_filter[test])['wallTime'])

for test in with_indexes:
    wall_time['withIndexes'] += np.sum(pd.DataFrame(with_indexes[test])['wallTime'])

print('Version delta time (BloomFiler-Indexes): ' + 
        f"{round(wall_time['withIndexes'] - wall_time['withBloomFilter']) / 10 ** 9} seconds")

Total execution time: 5:31:54
Version delta time (BloomFiler-Indexes): 703.355039628 seconds
